In [5]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
#norm.pdf(1, loc=0, scale=1)
#norm.cdf(1, loc=0, scale=1)



#gurobi key
options = {
    "WLSACCESSID": "26723063-59e7-4db9-a398-64f6ecd04b77",
    "WLSSECRET": "ab43d24e-de8d-46d9-9653-7e29d5de83e6",
    "LICENSEID": 2465782,
}

In [96]:
# data preparation
#set
M,N= 35, 15 
I=list(range(M))
J=list(range(N))


#parameter
headway_sched=6
sigma_sched=2

# delta_stop= np.random.normal(loc=3, scale=1, size=20)
# delta_stop[delta_stop < 1] = 1
# delta_stop = np.round(delta_stop).astype(int).tolist()
delta_stop=[2, 1, 3, 4, 2, 3, 4, 3, 3, 4, 3, 2, 2, 4, 3, 2, 2, 5, 3, 2, 3, 3, 3, 2, 1, 2, 3, 2, 1, 2, 4, 3, 4, 5, 3, 4, 3, 3, 4, 3, 3, 2, 3, 2, 4, 2, 2, 2, 1, 2, 2, 3, 3, 3, 5, 4, 4, 4, 3, 3, 2, 4, 3, 4, 2, 2, 2, 3, 3, 3, 3, 2, 2, 5, 1, 3, 2, 5, 3, 3, 3, 1, 3, 4, 1, 3, 4, 4, 4, 4, 4, 2, 4, 4, 3, 3, 2, 4, 2, 2, 1, 3, 2, 3, 1, 1, 5, 3, 1, 3, 2, 2, 4, 5, 1, 3, 2, 3, 2, 3, 4, 3, 1, 2, 3, 2, 4, 2, 2, 3, 3, 5, 2, 2, 5, 3, 3, 2, 3, 3]
pos_stop=[0]
i=0
while i<M-1:
    pos_stop.append(pos_stop[-1]+delta_stop[i])
    i+=1


# N_w=np.random.normal(loc=20, scale=12, size=M)
# N_w[N_w < 0] = 0
# N_w = np.round(N_w).astype(int).tolist()
N_w=[31, 30, 20, 29, 20, 18, 18, 20, 29, 14, 0, 14, 27, 8, 19, 1, 30, 17, 30, 26, 19, 26, 2, 7, 22, 10, 25, 8, 4, 0, 32, 14, 0, 36, 29, 11, 19, 1, 11, 8, 9, 21, 0, 42, 18, 21, 9, 7, 2, 1, 15, 0, 26, 7, 17, 45, 32, 10, 29, 26, 29, 19, 8, 19, 18, 28, 20, 34, 15, 24, 9, 19, 23, 14, 33, 35, 36, 12, 18, 18, 16, 29, 20, 11, 20, 0, 8, 25, 33, 35, 25, 8, 7, 19, 53, 22, 34, 34, 22, 37, 31, 20, 8, 35, 0, 30, 38, 27, 11, 15, 2, 27, 9, 38, 19, 29, 26, 18, 3, 10, 32, 26, 10, 28, 12, 21, 55, 16, 23, 24, 15, 41, 5, 12, 21, 3, 27, 17, 5, 21]
N_w=N_w[:M]
capacity=50
N_f=[0 if i <capacity else i-capacity  for i in N_w]
N_w=[50 if i>capacity else i for i in N_w]

control_stop=np.array(N_w).argsort().tolist()[::-1][:][:7]


In [ ]:
#MP formulation


with gp.Env(params=options) as env, gp.Model(env=env) as model:
    # parameter
    Sigma=model.addVars(J[0:], lb=0, name="Sigma")
    Mu=model.addVars(J[0:], lb=0,  name="Mu")

    sigma=model.addVars(J, lb=0, name="sigma")
    mu=model.addVars(J, lb=0,  name="mu")
